### Download HRRR model for inversion event
### 

## Plotting high-resolution data from HRRR in the Basin during inversion events

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rc
import xarray as xr
import numpy as np

import cartopy.feature as cfeature
from cartopy import crs

from herbie import  FastHerbie, Herbie

import time
import warnings

#little helper func
def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)

font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 12}

rc('font', **font)


#I will put some PRs into Herbie, we get a bunch of warnings with the regex calls
warnings.filterwarnings(action='ignore')

In [2]:
tbegin = time.time() #Timing

mt = "2023-02-06 06:00"
nfcst = 1
nprocs = 60
source = 'aws'

In [3]:
#Regex query string. 
#See https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfnatf02.grib2.shtml
# qstr = "(:TMP:|:PRES:|:HGT:|:UGRD:|:VGRD:|:SPFH:)"
# qstr = "(:TMP:|:PRES:|:SPFH:)"
# qstr = "(:CIMIXR:)"
qstr = "(:TMP:)"
FH = FastHerbie([mt], model="hrrr",product="nat",source=source, fxx=range(1,2))

#Download the subsets contianing the cloud fields. This can be 5 mins to 30 mins (ANL, Courtyard Marriott)
FH.download(qstr, max_threads=nprocs)
print('Downloaded')

#Load downloaded fields into a xarray dataset
ds_clouds = FH.xarray(qstr, remove_grib=True)
display(ds_clouds)

👨🏻‍🏭 Created directory: [/Users/johnlawson/data/hrrr/20230206]
Downloaded
Note: Returning a list of [3] xarray.Datasets because cfgrib opened with multiple hypercubes.


AttributeError: 'list' object has no attribute 'step'

In [ ]:
#Fix metadata
ds_clouds = ds_clouds.rename({'unknown' : 'cloudice',})
ds_clouds.cloudice.attrs['units'] = 'kg/kg'
ds_clouds.cloudice.attrs['long_name'] = 'Cloud ice mixing ratio'

tcloud = time.time() #timing

In [ ]:
FH = FastHerbie([mt], model="hrrr", fxx=[1,], source=source)

#Download temperature and Dewpoint at 2m
FH.download("(TMP|DPT):2 m", max_threads=nprocs)

#Load it into xarray
ds_t_and_dp = FH.xarray("(TMP|DPT):2 m", remove_grib=True)

ttemps = time.time()

In [ ]:
FH = FastHerbie([mt], model="hrrr", product="sfc",fxx=[1,], source=source)

FH.download("(:SNOD:)", max_threads=nprocs)
ds_snod = FH.xarray("(:SNOD:)", remove_grib=True)

tsnod = time.time()

In [ ]:
#Get vertical axis
FH = FastHerbie([mt], model="hrrr",product="nat",source=source, fxx=[0])
FH.download(':HGT:.*hybrid', max_threads=1) 
ds_heights = FH.xarray(':HGT:.*hybrid', remove_grib=True)

taxis = time.time() # timing

In [ ]:
# ds_precip = xr.merge([ds_rain, ds_clouds, ds_t_and_dp])
ds_precip = xr.merge([ds_clouds, ds_t_and_dp, ds_snod])
ds_precip['gph_zero_time'] = ds_heights.gh

#lets see what we have!
ds_precip

In [ ]:
#HRRR gives kg per m^2.. this is basically mm of rainfall. So convert to inches for our American audience 
inches_per_mm = 0.0393701

#The time step to show (hr 30 of 47). And some title text
timestep = 1
# tstr = f"{ds_precip.model.upper()}: {ds_precip.description}\nValid: {ds_precip.valid_time[timestep].dt.strftime('%H:%M UTC %d %b %Y').item()}"

locations_of_interest = [(-109.5287,40.4555),(-110.4029,40.1633),(-109.6774,40.0891)]
names_of_interest = ["Vernal","Duchesne","Ouray"]

#Map parameters
my_transform = crs.PlateCarree()
my_extent = [-110.6,-108.7,41.05,39.65]
nlines = 5
lonlines = trunc(np.linspace(my_extent[0], my_extent[1], nlines), 1)
latlines = trunc(np.linspace(my_extent[2], my_extent[3], nlines), 1)

coast = cfeature.NaturalEarthFeature(category='physical', scale='10m',
                            edgecolor='black', name='coastline')

counties = cfeature.NaturalEarthFeature(category='cultural', scale='10m',
                            edgecolor='black', name='admin_2_counties_lakes', alpha=.2)

In [ ]:
#Make a two panel figure
fig, (ax1, ax2) = plt.subplots( ncols=2, figsize=[18,8], constrained_layout=True,
                  subplot_kw={'projection' : ds_precip.herbie.crs})


#Plot 2m temperature
pc_temp = ax1.pcolormesh(ds_precip.longitude,  ds_precip.latitude, 
                         # ds_precip.isel(step=timestep).t2m-273.15, 
                         ds_precip.t2m-273.15, 

                         # cmap=cm_colorblind.HomeyerRainbow,
                        transform=my_transform)

c1 = plt.colorbar(pc_temp, fraction=0.046, pad=0.04)
c1.set_label(label='2m Temperature (Celcius)', size=18, weight='bold')
c1.ax.tick_params(labelsize=18) 

# plot_2 = "precip"
plot_2 = "snod"
if plot_2 == "precip":
    pc_precip = ax2.pcolormesh(ds_precip.longitude,  ds_precip.latitude, 
                             ds_precip.isel(step=timestep).precip_accum_1hr*inches_per_mm, 
                               cmap=cm_colorblind.HomeyerRainbow,
                            transform=my_transform, vmin = 0, vmax=.1)
    
    c2 = plt.colorbar(pc_precip, fraction=0.046, pad=0.04)
    c2.set_label(label='1hr Precip Accum (rain or SWE, Inches)', size=18, weight='bold')
    c2.ax.tick_params(labelsize=18)
else:
    pc_precip = ax2.pcolormesh(ds_precip.longitude,  ds_precip.latitude, 
                             # ds_precip.isel(step=timestep).snod, 
                             ds_precip.sde, 
                             # cmap=cm_colorblind.HomeyerRainbow,
                            transform=my_transform, vmin = 0, vmax=.1)
    
    c2 = plt.colorbar(pc_precip, fraction=0.046, pad=0.04)
    c2.set_label(label='Snow depth (m)', size=18, weight='bold')
    c2.ax.tick_params(labelsize=18)


#Zoom
ax1.set_extent(my_extent, crs=my_transform)
ax2.set_extent(my_extent, crs=my_transform)

#Gridlines
gl1 = ax1.gridlines(xlocs=lonlines, ylocs=latlines, x_inline=False, rotate_labels=False)
gl2 = ax2.gridlines(xlocs=lonlines, ylocs=latlines, x_inline=False, rotate_labels=False)
gl1.xlabels_bottom = True
gl1.ylabels_left = True
gl2.xlabels_bottom = True
gl2.ylabels_left = False
gl1.xlabel_style = {'size': 18}
gl1.ylabel_style = {'size': 18}
gl2.xlabel_style = {'size': 18}
gl2.ylabel_style = {'size': 18}
# ax1.set_title( tstr, loc="left")
# ax2.set_title( tstr, loc="left")

#Map features
ax1.add_feature(cfeature.STATES, facecolor='none', edgecolor='black')
ax1.add_feature(coast, facecolor='none', edgecolor='black')
ax1.add_feature(counties, facecolor='none', edgecolor='black')
ax2.add_feature(cfeature.STATES, facecolor='none', edgecolor='black')
ax2.add_feature(coast, facecolor='none', edgecolor='black')
ax2.add_feature(counties, facecolor='none', edgecolor='black')

#Plot Locations
ax1.scatter([loc[0] for loc in locations_of_interest],
            [loc[1] for loc in locations_of_interest], transform=my_transform, marker='o', color='r')

for i in range(len(names_of_interest)):
    ax1.text(locations_of_interest[i][0],locations_of_interest[i][1], 
             names_of_interest[i], transform = my_transform, size=15 )
    
ax2.scatter([loc[0] for loc in locations_of_interest],
            [loc[1] for loc in locations_of_interest], transform=my_transform, marker='o', color='r')

for i in range(len(names_of_interest)):
    ax2.text(locations_of_interest[i][0],locations_of_interest[i][1], 
             names_of_interest[i], transform = my_transform, size=15 )
    
    


In [ ]:
fig, (ax1, ax2) = plt.subplots( ncols=2, figsize=[18,8], constrained_layout=True,
                  subplot_kw={'projection' : ds_precip.herbie.crs})

pc_cw = ax1.pcolormesh(ds_precip.longitude,  ds_precip.latitude, 
                         ds_precip.cloudwater.isel(step=timestep).max(dim='hybrid'), 
                       # cmap=cm_colorblind.HomeyerRainbow,
                        transform=my_transform, vmax=0.002)

c1 = plt.colorbar(pc_cw, fraction=0.046, pad=0.04)
c1.set_label(label='Col. Max Cloud Water Mixing Ratio kg/kg', size=18, weight='bold')
c1.ax.tick_params(labelsize=18) 

pc_ci = ax2.pcolormesh(ds_precip.longitude,  ds_precip.latitude, 
                         ds_precip.cloudice.isel(step=timestep).max(dim='hybrid'), 
                       # cmap=cm_colorblind.HomeyerRainbow,
                        transform=my_transform, vmax=1e-5)

c2 = plt.colorbar(pc_ci, fraction=0.046, pad=0.04)
c2.set_label(label='Col. Max Cloud Ice Mixing Ratio kg/kg', size=18, weight='bold')
c2.ax.tick_params(labelsize=18) 



ax1.set_extent(my_extent, crs=my_transform)
ax2.set_extent(my_extent, crs=my_transform)

gl1 = ax1.gridlines(xlocs=lonlines, ylocs=latlines, x_inline=False, rotate_labels=False)
gl2 = ax2.gridlines(xlocs=lonlines, ylocs=latlines, x_inline=False, rotate_labels=False)

gl1.xlabels_bottom = True
gl1.ylabels_left = True
gl2.xlabels_bottom = True
gl2.ylabels_left = False

gl1.xlabel_style = {'size': 18}
gl1.ylabel_style = {'size': 18}

gl2.xlabel_style = {'size': 18}
gl2.ylabel_style = {'size': 18}


ax1.set_title( tstr, loc="left")
ax2.set_title( tstr, loc="left")

ax1.add_feature(cfeature.STATES, facecolor='none', edgecolor='black')
ax1.add_feature(coast, facecolor='none', edgecolor='black')
ax1.add_feature(counties, facecolor='none', edgecolor='black')

ax2.add_feature(cfeature.STATES, facecolor='none', edgecolor='black')
ax2.add_feature(coast, facecolor='none', edgecolor='black')
ax2.add_feature(counties, facecolor='none', edgecolor='black')

ax1.scatter([loc[0] for loc in locations_of_interest],
            [loc[1] for loc in locations_of_interest], transform=my_transform, marker='o', color='r')

for i in range(len(names_of_interest)):
    ax1.text(locations_of_interest[i][0],locations_of_interest[i][1], 
             names_of_interest[i], transform = my_transform, size=15 )
    
ax2.scatter([loc[0] for loc in locations_of_interest],
            [loc[1] for loc in locations_of_interest], transform=my_transform, marker='o', color='r')

for i in range(len(names_of_interest)):
    ax2.text(locations_of_interest[i][0],locations_of_interest[i][1], 
             names_of_interest[i], transform = my_transform, size=15 )

In [ ]:
dsi = ds_precip.herbie.nearest_points(locations_of_interest, 
                               names=names_of_interest)
dsi

In [ ]:
#Make a figure with four rows. 
fig, (ax1, ax2, ax3, ax4)  = plt.subplots(nrows = 4, sharex=True, constrained_layout=True, figsize=[15,15])


#Use GPH at t=0 as the vertical axis and plot cloud ice and cloud water for one site
dsi.isel(point=0).set_index(hybrid="gph_zero_time").cloudice.plot(x="valid_time", vmin=0, vmax=5e-6, 
                                                        # cmap = cm_colorblind.HomeyerRainbow, ax=ax1
                                                                  )


dsi.isel(point=0).set_index(hybrid="gph_zero_time").cloudwater.plot(x="valid_time", vmin=0, vmax=0.0005, 
                                                        # cmap = cm_colorblind.HomeyerRainbow, ax=ax2
                                                                    )

acpc = False
if acpc:
    #Plot precip 1hr accums for all sites
    (dsi.precip_accum_1hr*inches_per_mm).isel(point=2).plot(color='red', 
                                                          x="valid_time", marker=".", 
                                                          label=dsi.point[2].values,
                                                         ax=ax3)
    
    
    (dsi.precip_accum_1hr*inches_per_mm).isel(point=0).plot(color='blue', 
                                                          x="valid_time", marker=".", 
                                                          label=dsi.point[0].values,
                                                         ax = ax3)
    
    (dsi.precip_accum_1hr*inches_per_mm).isel(point=1).plot(color='purple', 
                                                          x="valid_time", marker=".",
                                                          label=dsi.point[1].values,
                                                         ax=ax3)


#plot temperature and dewpoints for all sites. 
(dsi.t2m - 273.15).isel(point=0).plot(color='blue', x="valid_time", marker=".", label=dsi.point[0].values, ax=ax4)
(dsi.t2m - 273.15).isel(point=1).plot(color='purple', x="valid_time", marker=".", label=dsi.point[1].values, ax=ax4)
(dsi.t2m - 273.15).isel(point=2).plot(color='red', x="valid_time", marker=".", label=dsi.point[2].values, ax=ax4)

(dsi.d2m - 273.15).isel(point=0).plot(color='blue', x="valid_time", marker=".", linestyle='dashed', ax=ax4)
(dsi.d2m - 273.15).isel(point=1).plot(color='purple', x="valid_time", marker=".", linestyle='dashed', ax=ax4)
(dsi.d2m - 273.15).isel(point=2).plot(color='red', x="valid_time", marker=".", linestyle='dashed', ax=ax4)


#Embelish
ax4.set_ylabel('Temperature and dew point (celcius)')
ax4.set_xlabel('Time and date (UTC)')
ax4.set_title('HRRR Temperature and dewpoint forecasts')


plt.legend()

ax1.set_ylim([0, 15000])
ax2.set_ylim([0, 15000])
#ax3.set_ylim([0,.5])
plt.grid()
ax3.set_ylabel('Rain Rate (in/hr)')
ax3.set_xlabel('Time and date (UTC)')
ax3.set_title('HRRR rain rate forecasts')


In [ ]:
# Convert temperature to potential temperature
# Need to think about moisture and using theta-e?
# theta = T(1000/P)**(R/Cp)

def calculate_moist_stability_indices(T, P, z, u, v, q):
    g = 9.81  # Acceleration due to gravity
    Rd = 287  # Gas constant for dry air
    cp = 1005  # Specific heat at constant pressure
    Lv = 2.5e6  # Latent heat of vaporization
    P0 = 1000  # Reference pressure

    # Calculate potential temperature
    theta = T * (P0 / P)**(Rd / cp)

    # Calculate equivalent potential temperature
    theta_m = theta * np.exp((Lv * q) / (cp * T))

    # Calculate vertical gradients
    dtheta_m_dz = np.gradient(theta_m, z, axis=0)
    du_dz = np.gradient(u, z, axis=0)
    dv_dz = np.gradient(v, z, axis=0)

    # Calculate moist Brunt-Vaisala frequency squared (N^2_m)
    N2_m = (g / theta_m) * dtheta_m_dz

    # Calculate Richardson number (Ri)
    Ri = N2_m / (du_dz**2 + dv_dz**2)

    return N2_m, Ri

# Dummy data (replace with your actual data)
T = np.random.rand(10, 10, 10)  # Temperature in Kelvin
P = np.random.rand(10, 10, 10)  # Pressure in hPa
z = np.linspace(0, 3000, 10)  # Height in meters
u = np.random.rand(10, 10, 10)  # u wind component
v = np.random.rand(10, 10, 10)  # v wind component
q = np.random.rand(10, 10, 10)  # Specific humidity

# Calculate moist stability indices
N2_m, Ri = calculate_moist_stability_indices(T, P, z, u, v, q)


In [ ]:
qstr = '(' 
for i in range(nfcst):
    if i != nfcst -1:
        qstr = qstr +  f':APCP:surface:{i}-{i+1} ho*|'
    else:
        qstr = qstr +  f':APCP:surface:{i}-{i+1} ho*)'

H = Herbie(
    mt,  # model run date
    model="hrrr",  # model name
    product="sfc",  # model produce name (model dependent)
    fxx=1,  # forecast lead time
)

#Lets test!
H.read_idx(searchString=qstr)

In [ ]:
#Download
FH = FastHerbie([mt], model="hrrr",source=source, fxx=[1,])
FH.download(qstr, max_threads=nprocs) #This needs a good (and cheap) internet connection... ~1m on my connection

#Read to xarray
ds_rain = FH.xarray(qstr, remove_grib=True)

#Add a timestep at start
ds_rain = xr.concat([ds_rain.isel(step=0), ds_rain], dim='step')
ds_rain['step'] = ds_clouds['step']
ds_rain['valid_time'] = ds_clouds['valid_time']
ds_rain = ds_rain.rename({'tp': 'precip_accum_1hr'})

#set time zero to zero mm/hr
ds_rain.precip_accum_1hr[0, :, :] =  ds_rain.precip_accum_1hr[0, :, :]*0.0

tprecip = time.time() #timing